# ⚫ İki Kara Deliğin Birleşmesi (Binary Black Hole Merger) Simülasyonu

Bu proje, **Python + Matplotlib** kullanılarak iki kara deliğin **inspiral → merger → ringdown** sürecini görselleştirir.  
Sağ tarafta, birleşmeye bağlı olarak üretilen **gravitational wave (h₊) dalga formu** animasyonla eş zamanlı gösterilir.

---

## 🔹 Temel Fizik

İki kara delik (BH) sistemi, birbirine yakın konumda olduklarında kütleçekimsel dalgalar yayarak enerji kaybeder. Bu süreç üç aşamadan oluşur:

1. **Inspiral (yaklaşma)**  
   - Newtonian benzeri yörünge:  
     \[
     \omega^2 = \frac{M_{\rm tot}}{r^3}
     \]  
     Burada \(M_{\rm tot} = M_1 + M_2\), \(r\) ise BH merkezleri arası mesafedir.
   - Peters-Mathews formülü (enerji kaybına bağlı yavaşlama):
     \[
     \frac{dr}{dt} = -\frac{64}{5} \frac{\mu M_{\rm tot}^2}{r^3}, \quad \mu = \frac{M_1 M_2}{M_{\rm tot}}
     \]

2. **Merger (birleşme)**  
   - BH’ler çok yakınlaşınca tek bir remnant BH oluşur.  
   - Bu süreçte GW amplitüdü maksimuma çıkar (chirp).

3. **Ringdown (sönümleme)**  
   - Remnant BH, sönümlü salınım (quasi-normal mode) ile kararlı hale gelir:
     \[
     h(t) \sim e^{-t/\tau} \sin(\omega_0 t)
     \]

---

## 🔹 Gravitasyonel Dalga Formülü (h₊)

Kabaca, GW amplitüdü:
\[
h_+ \approx \frac{4 (M_c)^{5/3} \omega^{2/3}}{D} (1 + \cos^2 \iota) \cos(2\phi)
\]

- \(M_c = (\mu^{3/5} M_{\rm tot}^{2/5})\) → chirp kütlesi  
- \(\omega\) → yörüngesel açısal hız  
- \(D\) → gözlemci uzaklığı  
- \(\iota\) → gözlemci açısı (face-on: 0)  
- \(\phi\) → orbital faz  

Bu, Newtonian yaklaşımı ve quadrupole formülünün **ilk mertebe kabaca** çözümüdür.

---

## 🔹 Matematiksel Temel

### 1. Newtonian Kepler Yörüngesi
\[
\frac{d^2 \vec{r}}{dt^2} = - \frac{M_{\rm tot}}{r^3} \vec{r}
\]

### 2. Enerji Kaybı ve Inspiral
GW enerji kaybı:
\[
\frac{dE}{dt} = -\frac{32}{5} \frac{\mu^2 M_{\rm tot}^3}{r^5}
\]

Yörünge enerjisi \(E = -\frac{\mu M_{\rm tot}}{2 r}\) olduğu için:
\[
\frac{dr}{dt} = \frac{dE/dt}{dE/dr} = -\frac{64}{5} \frac{\mu M_{\rm tot}^2}{r^3}
\]

### 3. Ringdown
Remnant BH, sönümlü sinüs salınımı ile kararlı hale gelir:
\[
h(t) = h_0 e^{-t/\tau} \sin(\omega_0 t)
\]
- \(\tau\) → sönümleme zamanı  
- \(\omega_0\) → quasi-normal mode frekansı

---

## 🔹 Simülasyon Detayları

- **Kara Delik Boyutları:** Schwarzschild yarıçapı \( r_s = 2 M \)  
- **Başlangıç Ayrımı:** r₀  
- **Birleşme Eşiği:** r_merge  
- **Animasyon:** 3D kara delik + GW dalga formu  
- **Renkler:** BH tel kafesi beyaz, arka plan siyah, gözlemci yıldızları soluk beyaz  

---

## 🔹 Kullanılan Kısmi Basitleştirmeler

- Post-Newtonian veya tam genel görelilik kullanılmadı, Newtonian + sönümlü sinüs yaklaşımı.  
- Spin ve eksantriklik ihmal edildi.  
- GW amplitüdü yüzeysel: h₊ yalnızca yaklaşık chirp patterni verir.  

---

## 🔹 Gelecek Geliştirmeler

1. **Post-Newtonian düzeltmeler** eklenebilir.  
2. **Kerr BH spin etkisi** simülasyona dahil edilebilir.  
3. **Gözlemci açısına göre polarizasyon** ve hₓ dalgası görselleştirilebilir.  
4. **Gerçek zamanlı kütle kaybı ve birleşme sonrası remnant spin** hesaplanabilir.  

---

## 🔹 Referanslar

- Peters, P.C., & Mathews, J. (1963). *Gravitational Radiation from Point Masses in a Keplerian Orbit*. Phys. Rev., 131, 435  
- Misner, C.W., Thorne, K.S., & Wheeler, J.A. (1973). *Gravitation*  
- Maggiore, M. (2008). *Gravitational Waves: Volume 1: Theory and Experiments*  



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

# ===============================
# Fiziksel (basitleştirilmiş) ayarlar
# ===============================
# Doğal birimler: G = c = 1
M1 = 0.5         # Birinci BH kütlesi
M2 = 0.5         # İkinci BH kütlesi
Mtot = M1 + M2   # Toplam kütle
mu   = M1*M2/Mtot
D    = 50.0      # Gözlemci uzaklığı (göreli ölçek)

# Schwarzschild yarıçapları (r_s = 2M)
r_s1 = 2*M1
r_s2 = 2*M2

# Başlangıç ayrımı ve birleşme eşiği
r0       = 8.0          # başlangıç iki BH merkezi arası mesafe
r_merge  = 2.5          # birleşmeye geçtiğimizi söyleyeceğimiz eşik (yaklaşık)
k_rad    = (64.0/5.0)*mu*(Mtot**2)   # Peters-Mathews ~ a_dot = -k / a^3

# Animasyon ayarları
frames   = 600
interval = 30           # ms
dt       = 0.02         # zaman adımı (göreli)

# GW dalga formu için açısal bağımlılık (yüzümüze bakıyor gibi olsun)
iota = 0.0  # 0: face-on
plus_factor = (1 + np.cos(iota)**2)

# ===============================
# Yardımcı fonksiyonlar
# ===============================
def omega_kepler(r):
    # Newtonian Kepler: omega^2 = Mtot / r^3
    return np.sqrt(Mtot / (r**3))

def gw_amplitude(omega):
    # h_plus ~ (M_c)^(5/3) * (omega)^(2/3) / D  (kabaca)
    # M_c = chirp mass = mu^(3/5) * Mtot^(2/5)
    M_c = (mu**(3/5)) * (Mtot**(2/5))
    A0 = 4.0 * (M_c**(5/3)) / D
    return A0 * (omega**(2/3))

# ===============================
# Figür ve eksenler (3D + 2D waveform)
# ===============================
plt.rcParams['figure.facecolor'] = 'black'
plt.rcParams['axes.facecolor'] = 'black'
plt.rcParams['text.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'

fig = plt.figure(figsize=(11,6))

# Sol: 3D uzay
ax3d = fig.add_subplot(1,2,1, projection='3d')
ax3d.set_xlim(-6, 6)
ax3d.set_ylim(-6, 6)
ax3d.set_zlim(-6, 6)
ax3d.axis('off')

# Sağ: dalga formu (h+)
axw = fig.add_subplot(1,2,2)
axw.set_xlim(0, frames*dt)
axw.set_ylim(-0.02, 0.02)
axw.set_xlabel("zaman (arb. birim)")
axw.set_ylabel("h+ (arb.)")
axw.grid(alpha=0.2, linestyle=':')

# Kara delik küreleri (tel kafes + siyah yüzey)
phi, theta = np.mgrid[0:np.pi:40j, 0:2*np.pi:40j]

def make_sphere(R):
    xs = R * np.sin(phi) * np.cos(theta)
    ys = R * np.sin(phi) * np.sin(theta)
    zs = R * np.cos(phi)
    return xs, ys, zs

# İki BH başlangıç pozisyonu (eş kütleli: +/- r/2)
r = r0
theta_orb = 0.0

xs1, ys1, zs1 = make_sphere(r_s1)
xs2, ys2, zs2 = make_sphere(r_s2)

# 3D artist objeleri
surf1 = ax3d.plot_surface(xs1 - r/2, ys1, zs1, color='black', edgecolor=None, zorder=20)
wire1 = ax3d.plot_wireframe(xs1 - r/2, ys1, zs1, color='white', linewidth=0.5, rstride=5, cstride=5, zorder=25)
surf2 = ax3d.plot_surface(xs2 + r/2, ys2, zs2, color='black', edgecolor=None, zorder=20)
wire2 = ax3d.plot_wireframe(xs2 + r/2, ys2, zs2, color='white', linewidth=0.5, rstride=5, cstride=5, zorder=25)

# Birleşme sonrası tek BH (başta görünmez)
Mrem   = 0.95*Mtot
r_srem = 2*Mrem
xsr, ysr, zsr = make_sphere(r_srem)
surfR = ax3d.plot_surface(xsr, ysr, zsr, color='black', edgecolor=None, alpha=0.0, zorder=30)
wireR = ax3d.plot_wireframe(xsr, ysr, zsr, color='white', linewidth=0.6, rstride=5, cstride=5, alpha=0.0, zorder=35)

# Hafif yıldız arka planı
np.random.seed(7)
stars = (np.random.rand(300,3)-0.5)*2*12
ax3d.scatter(stars[:,0], stars[:,1], stars[:,2], s=2, c='white', alpha=0.25, zorder=1)

# Dalga formu çizgisi
t_vals = np.linspace(0, frames*dt, frames)
h_vals = np.zeros(frames)
(line_h,) = axw.plot([], [], lw=1.5)

# Kamera açısı
def set_view(f):
    elev = 25
    azim = (0.6 * f) % 360
    ax3d.view_init(elev=elev, azim=azim)

# ===============================
# Animasyon fonksiyonu
# ===============================
merger_started = False
ringdown_tau = 0.15  # sönüm zaman ölçeği
ringdown_omega0 = 12.0  # ringdown salınım frekansı (görsel)

def update(frame):
    global r, theta_orb, merger_started

    set_view(frame)

    # Inspiral (merger'a kadar): r' = -k / r^3
    if not merger_started:
        r = max(r - (k_rad * dt) / (r**3), r_merge)
        om = omega_kepler(r)
        theta_orb += om * dt

        # Eş kütle: BH1 ve BH2 +/- r/2 yarıçapında
        x1 = - (r/2) * np.cos(theta_orb)
        y1 = - (r/2) * np.sin(theta_orb)
        x2 = + (r/2) * np.cos(theta_orb)
        y2 = + (r/2) * np.sin(theta_orb)
        z1 = 0.0
        z2 = 0.0

        # Küreleri bu merkezlere taşı
        for obj in [surf1, wire1]:
            obj.remove()
        for obj in [surf2, wire2]:
            obj.remove()

        xs1, ys1, zs1 = make_sphere(r_s1)
        xs2, ys2, zs2 = make_sphere(r_s2)

        # BH1
        s1 = ax3d.plot_surface(xs1 + x1, ys1 + y1, zs1 + z1, color='black', edgecolor=None, zorder=20)
        w1 = ax3d.plot_wireframe(xs1 + x1, ys1 + y1, zs1 + z1, color='white', linewidth=0.5, rstride=5, cstride=5, zorder=25)
        # BH2
        s2 = ax3d.plot_surface(xs2 + x2, ys2 + y2, zs2 + z2, color='black', edgecolor=None, zorder=20)
        w2 = ax3d.plot_wireframe(xs2 + x2, ys2 + y2, zs2 + z2, color='white', linewidth=0.5, rstride=5, cstride=5, zorder=25)

        # Artist referanslarını güncelle
        globals()['surf1'], globals()['wire1'] = s1, w1
        globals()['surf2'], globals()['wire2'] = s2, w2

        # GW h_plus (chirp): 2*phi fazıyla
        A = gw_amplitude(om)
        h = plus_factor * A * np.cos(2*theta_orb)

        h_vals[frame] = h
        line_h.set_data(t_vals[:frame+1], h_vals[:frame+1])

        if r <= r_merge + 1e-6:
            merger_started = True

    else:
        # Merger + ringdown: tek BH görünür, iki BH kaybolur
        # Eski yüzeyleri sil
        for obj in [surf1, wire1, surf2, wire2]:
            try:
                obj.remove()
            except:
                pass

        # Remnant'ı yavaşça alfa ile ortaya çıkar
        alpha_now = min(1.0, (frame/50.0))
        wireR.set_alpha(0.6*alpha_now)
        surfR.set_alpha(1.0*alpha_now)

        # Ringdown dalga formu: sönümlü sinüs
        t = t_vals[frame]
        # h ~ e^{-t/tau} * sin(omega0 * t)
        h_vals[frame] = 0.012*np.exp(-(t - t_vals[np.argmax(h_vals)]) / ringdown_tau) * np.sin(ringdown_omega0 * (t - t_vals[np.argmax(h_vals)]))
        line_h.set_data(t_vals[:frame+1], h_vals[:frame+1])

    return (line_h,)

anim = FuncAnimation(fig, update, frames=frames, interval=interval, blit=False)

from IPython.display import HTML
# Büyük animasyon uyarısı gelirse aşağıyı açıp limiti arttır:
# import matplotlib as mpl
# mpl.rcParams['animation.embed_limit'] = 80  # MB
HTML(anim.to_jshtml())
